# My Personal Notes For Learning ChatGPT API
### https://github.com/mmrzax

In [57]:
import os
import openai
import requests
import json

In [2]:
# get openai api key from enviroment
openai.api_key = os.getenv("OPENAI_API_KEY")
# print(openai.api_key)

### Messages and Roles

The main input is the messages parameter. Messages must be an array of message objects, where each object has a role (either "system", "user", or "assistant") and content. Conversations can be as short as one message or many back and forth turns.

Typically, a conversation is formatted with a system message first, followed by alternating user and assistant messages.

The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."

The user messages provide requests or comments for the assistant to respond to. Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

In [ ]:
res1 = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are an engineer and assistant, skilled in control systems."},
    {"role": "user", "content": "In your opinion, what is the best part of industries that can be controlled by artificial intelligence?"}
  ]
)

In [3]:
res2 = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
        {"role": "user", "content": "Where was it played?"}
    ]
)

In [13]:
res3 = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "assistant", "content": "Persepolis lose the game to Kashima Antlers AFC Final"},
        {"role": "user", "content": "Where was it played?"}
    ]
)

**دلیل آوردن یک سری مکالمات قبل از پرامپت اصلی اینه که چت بات بتونه بفهمه اصلا منظور ما از سوال آخر یعنی بازیه کجا پلی شده کدوم بازی هست<br>در کل این تاریخچه چت خیلی مهمه و هر بار باید سیوش کرد و اگه به محدودیت سایز توکن رسیدیم باید از شکل کوتاه شده استفاده کنیم در این [لینک](https://platform.openai.com/docs/guides/gpt-best-practices/tactic-for-dialogue-applications-that-require-very-long-conversations-summarize-or-filter-previous-dialogue) توضیح داده شده**

**در اینجا میخواهیم به بات بگیم که از یه تابع کمکی برای پاسخ دادن به کاربر استفاده کنه، برای مثال در اینجا یک تابع کمکی که با یک ریکوئست دمای هوا رو میگیره نوشتم**

In [68]:
def get_current_weather(location: str, unit: str = "celcius"):
    units: dict[str, str] = {"celsius": "m", "fahrenheit": "u"}
    r = requests.get(f"https://wttr.in/{location}?{units.get(unit)}&format=3")
    return r.text
print(get_current_weather("Rafsanjan"))

Rafsanjan: ☀️   +16°C



In [64]:
messages = [{"role": "user", "content": "آب و هوا اکنون در mashhad چگونه است؟"}]
functions = [{
    "name": "get_current_weather",
    "description": "آب و هوا کنونی را برای شهر داده شده دریافت کن",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "نام شهر به زبان انگلیسی، برای مثال Tehran",
            },
            "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
        },
        "required": ["location"],
    }
}]
res4 = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions,
    function_call="auto",
)
response_message = res4["choices"][0]["message"]

In [65]:
# Step 2: check if GPT wanted to call a function
if response_message.get("function_call"):
    # Step 3: call the function
    # Note: the JSON response may not always be valid; be sure to handle errors
    available_functions = {
        "get_current_weather": get_current_weather,
    }  # only one function in this example, but you can have multiple
    function_name = response_message["function_call"]["name"]
    function_to_call = available_functions[function_name]
    function_args = json.loads(response_message["function_call"]["arguments"])
    function_response = function_to_call(
        location=function_args.get("location"),
        unit=function_args.get("unit"),
    )

    # Step 4: send the info on the function call and function response to GPT
    messages.append(response_message)  # extend conversation with assistant's reply
    messages.append(
        {
            "role": "function",
            "name": function_name,
            "content": function_response,
        }
    )  # extend conversation with function response
    second_response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
    )  # get a new response from GPT where it can see the function response
    print(second_response)


{
  "id": "chatcmpl-8GBpdEUezKkIIENJgexEmoeyy2OdG",
  "object": "chat.completion",
  "created": 1698869421,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "\u062f\u0631 \u0645\u0634\u0647\u062f \u0647\u0645 \u0627\u06a9\u0646\u0648\u0646 \u0622\u0628 \u0648 \u0647\u0648\u0627 \u0635\u0627\u0641 \u0648 \u0622\u0641\u062a\u0627\u0628\u06cc \u0627\u0633\u062a \u0648 \u062f\u0645\u0627\u06cc \u0647\u0648\u0627 \u062d\u062f\u0648\u062f 11 \u062f\u0631\u062c\u0647 \u0633\u0627\u0646\u062a\u06cc\u06af\u0631\u0627\u062f \u0627\u0633\u062a."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 70,
    "completion_tokens": 54,
    "total_tokens": 124
  }
}


In [66]:
print(second_response['choices'][0]['message']['content'])

در مشهد هم اکنون آب و هوا صاف و آفتابی است و دمای هوا حدود 11 درجه سانتیگراد است.
